In this file I look at the errors on the spectra measurements, to see if they are correct.

In [2]:
# Import calibration functions
# Run this to reimport without having to restart the whole notebook
import radial_velocity_functions
import importlib
importlib.reload(radial_velocity_functions)
from radial_velocity_functions import *

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
import tqdm as tqdm
from tqdm import tqdm
import pandas as pd
from datetime import datetime
from scipy.interpolate import interp1d

# plt styles
matplotlib.rcParams["text.usetex"] = True
matplotlib.rcParams["font.family"] = "serif"
matplotlib.rcParams["font.size"] = 10 
matplotlib.rcParams['figure.dpi'] = 150

from os import walk

First compute chi2 for different error factors

In [ ]:
N_processes = 6

# Get list of files and find all features
# filenames = get_spectra_filenames_without_duplicate_dates(SPECTRA_PATH_34411)
filenames = get_all_spectra_filenames(SPECTRA_PATH_34411)

# Find features in all files 
features = []
print("Finding features for all files...")
for filename in tqdm(filenames):
    features.append(find_features(filename, log=False))


# Setup coords :: Compute one list of coords
all_coords = compute_coords(filenames)        
        
# Define function for each process
def compute(coords):
    x = coords[0]
    y = coords[1]
    matches = find_feature_matches2(features[x], features[y], log=False)
    shifts = compute_all_feature_shifts(matches, log=False, return_extra=True)
    return shifts


if __name__ == '__main__':
    import multiprocess
    pool = multiprocess.Pool(processes = N_processes)

    # With progress bar
    result = []
    print("Computing shifts for all files combinations...")
    for r in tqdm(pool.imap_unordered(compute, all_coords), total=len(all_coords)):
        result.append(r)
        
    # Convert to DataFrame
    df = pd.DataFrame(np.vstack(result))
    df.columns = ["i", "j", "rv [m/s]", "σ_rv [m/s]", "χ2", "λ_i [Å]", "λ_interp_i [Å]", "pixel_i", "order_i", "time_i", "λ_j [Å]", "λ_interp_j [Å]", "pixel_j", "order_j", "time_j"]
    
    # Sort by i and j (needed because we compute in paralel)
    df = df.sort_values(['i', 'j', 'λ_i [Å]']).reset_index(drop=True)

    # Save
    df.to_csv("data_files_for_simon/all_files_all_obs_k_feature_fits.csv", index=False)
    print("Finished!")
